Now the model has been trained, and what's next is the post-hoc analysis of performance. In particular, does it at least predict % changes of the correct sign? If it can do that correctly, this model is bad, but at least useful.

Load model
Load test data

FP, see what predictions are

Compare

In [1]:
import sys
sys.path.append('../src')

In [2]:
from train import CryptoTrainer
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
trader = CryptoTrainer.load_from_checkpoint("/Users/noahkasmanoff/Desktop/F21/jetcoin/src/jetcoin-src/dw8p9e5n/checkpoints/epoch=17-step=1151.ckpt")
                                            
                                            

In [7]:
trader.state_dict()

torch.save(trader.state_dict(), '../model/saved.ckpt')

In [8]:
trader = CryptoTrainer.load_from_checkpoint("../model/saved.ckpt")


TypeError: __init__() missing 5 required positional arguments: 'args', 'n_layers', 'filter_size', 'dropout_rate', and 'resolution'

In [10]:
model = torch.load("../model/saved.ckpt")

In [11]:
model

OrderedDict([('model.inv_timescales',
              tensor([1.0000e+00, 4.3288e-01, 1.8738e-01, 8.1113e-02, 3.5112e-02, 1.5199e-02,
                      6.5793e-03, 2.8480e-03, 1.2328e-03, 5.3367e-04, 2.3101e-04, 1.0000e-04])),
             ('model.decoder.layers.0.self_attention_norm.weight',
              tensor([0.9965, 0.9878, 1.0044, 0.9945, 0.9967, 0.9881, 1.0021, 0.9880, 1.0000,
                      0.9945, 0.9969, 1.0021, 0.9983, 1.0092, 1.0058, 1.0002, 0.9896, 0.9840,
                      0.9892, 0.9928, 0.9909, 1.0003, 0.9988, 0.9959])),
             ('model.decoder.layers.0.self_attention_norm.bias',
              tensor([ 0.0045, -0.0035,  0.0018,  0.0004,  0.0022, -0.0005, -0.0040,  0.0067,
                      -0.0031,  0.0029, -0.0003, -0.0104, -0.0022,  0.0044,  0.0044,  0.0027,
                      -0.0036, -0.0119, -0.0032, -0.0057, -0.0012,  0.0031,  0.0032,  0.0021])),
             ('model.decoder.layers.0.self_attention.linear_q.weight',
              tensor([

In [4]:
from dataloaders import create_dataloaders

In [5]:
# TODO -save all these args to model, add to trader. for loading loaders
train_loader, valid_loader, test_loader, today_loader, approx_resolution = create_dataloaders(crypto=trader.hparams.args.crypto,
                                                                            values=trader.hparams.args.values,
                                                                            batch_size=1,
                                                                            labels_to_load=trader.hparams.args.labels_to_load.split(','),
                                                                        prior_years = trader.hparams.args.prior_years,
                                                                            prior_days = trader.hparams.args.prior_days,
                                                                            window = trader.hparams.args.window_size,
                                                                                buy_thresh=trader.hparams.args.buy_thresh)

In [6]:
y = torch.randn(1)

In [7]:
if torch.cuda.is_available():
    trader.cuda(); # if available!

trader.eval();
#with torch.no_grad():
for price, price_norm, date, pct_change in today_loader:
    if torch.cuda.is_available():
        price_norm = price_norm.cuda()# if available!

#        print(trader.forward(price).item(),'||', pct_change.item())
    y_pred = trader.forward(price_norm)#.item()


In [8]:
from torch.nn.functional import mse_loss

In [ ]:
torch.randn(1)

In [9]:
loss = mse_loss(y_pred,y) 

In [10]:
y_pred, loss

(tensor([[0.2046]], grad_fn=<AddmmBackward>),
 tensor(0.3730, grad_fn=<MseLossBackward>))

In [ ]:
y_pred, loss

In [11]:
from torch.optim import SGD

In [12]:
optimizer =  SGD(trader.parameters(),lr=trader.learning_rate)

In [22]:
optimizer = torch.optim.SGD(trader.parameters(), lr=trader.learning_rate, weight_decay=trader.weight_decay
optimizer.zero_grad()
loss = mse_loss(y_pred,torch.randn(1)) 
loss.backward()
optimizer.step()

SyntaxError: invalid syntax (<ipython-input-22-ea940c22c446>, line 2)

In [19]:
# repeat, did the model's parameters actually update? 
if torch.cuda.is_available():
    trader.cuda(); # if available!

trader.eval();
#with torch.no_grad():
for price, price_norm, date, pct_change in today_loader:
    if torch.cuda.is_available():
        price_norm = price_norm.cuda()# if available!

#        print(trader.forward(price).item(),'||', pct_change.item())
    y_pred = trader.forward(price_norm)#.item()


In [20]:
loss = mse_loss(y_pred,y) 

In [17]:
y ,y_pred, loss

(tensor([0.8154]),
 tensor([[-1.4949]], grad_fn=<AddmmBackward>),
 tensor(5.3374, grad_fn=<MseLossBackward>))

In [21]:
y ,y_pred, loss

(tensor([0.8154]),
 tensor([[0.1556]], grad_fn=<AddmmBackward>),
 tensor(0.4352, grad_fn=<MseLossBackward>))

In [ ]:
y_pred = []
y_true = []
trader.eval();

with torch.no_grad():
    for price, pct_change in today_loader:

#        print(trader.forward(price).item(),'||', pct_change.item())
        y_pred.append(trader.forward(price).item())
        y_true.append(pct_change.item())

In [ ]:
plt.title("Percent Change in Price on Following Day")
plt.plot(y_pred,label='predicted')
plt.plot(y_true,label='true')
plt.axhline()

Graph notes... 

What about today? 

In [ ]:
sys.path.append('../dat/')
from collect_and_prepare_data import prepare_data

In [ ]:
_, _, coin_test = prepare_data(prior_years=5,crypto='bitcoin',values='usd', buy_thresh = .05, window = 28)

In [ ]:
coin_test[-1]['date']

In [ ]:
trader.forward(torch.Tensor(coin_test[-1]['prices']).unsqueeze(0))

In [ ]:
def get_today_input():
    """
    Return the input for today's price history for the predicted pct_change. 
    """
    
    

In [ ]:

    cg = CoinGeckoAPI()

    today = cg.get_price(ids='bitcoin', vs_currencies='usd',include_last_updated_at=True)['bitcoin']['last_updated_at']
    start = int(today - prior_years*31536000) # subtract prior years


    btc_history = cg.get_coin_market_chart_range_by_id(id=crypto,vs_currency=values,include_market_cap='true',from_timestamp= start,to_timestamp=today)
    prices = np.array(btc_history['prices'])
    timestamps = prices[:,0]
    price = prices[:,1]
    dates = [datetime.fromtimestamp(timestamp // 1000) for timestamp in timestamps]
    cg_df = pd.DataFrame()
    cg_df['timestamp'] = timestamps
    cg_df['date'] = dates
    cg_df[crypto + '_price'] = price

In [ ]:
from datetime import datetime
from pycoingecko import CoinGeckoAPI
import numpy as np
import pandas as pd

In [ ]:
cg = CoinGeckoAPI()

today = cg.get_price(ids='bitcoin', vs_currencies='usd',include_last_updated_at=True)['bitcoin']['last_updated_at']
start = int(today - 5*31536000) # subtract prior years
crypto='bitcoin'

In [ ]:

btc_history = cg.get_coin_market_chart_range_by_id(id='bitcoin',vs_currency='usd',include_market_cap='true',from_timestamp= start,to_timestamp=today)
prices = np.array(btc_history['prices'])
timestamps = prices[:,0]
price = prices[:,1]
dates = [datetime.fromtimestamp(timestamp // 1000) for timestamp in timestamps]
cg_df = pd.DataFrame()
cg_df['timestamp'] = timestamps
cg_df['date'] = dates
cg_df[crypto + '_price'] = price


In [ ]:
coin_df = cg_df

In [ ]:
window = 28
coin_df['moving_avg'] = coin_df['bitcoin_price'].rolling(window=window).mean().shift(1) # for day i, computes the window rolling average for the prior i-1 to i-1-window days.
coin_df['pct_change'] =  (coin_df['bitcoin_price'] - coin_df['moving_avg']) / coin_df['moving_avg']


In [ ]:
coin_df.loc[coin_df['date'].astype(str).str.contains('2021-01-01')]

In [ ]:
i = 3
prices = coin_df['bitcoin_price'].values[i-window:i]

In [ ]:
prices

In [ ]:
coin_df

In [ ]:
coin_df.loc[coin_df['pct_change'] == coin_test[-1]['pct_change']]

In [ ]:
i = 364
prices = coin_df['bitcoin_price'].values[i-window:i]

In [ ]:
prices

In [ ]:
pct_change = coin_df['pct_change'].values[i]

In [ ]:
pct_change

In [ ]:
coin_test[-1]

In [ ]:
coin_df

In [ ]:
coin_df['bitcoin_price'].values[coin_df.shape[0]-window:coin_df.shape[0]+1]